<a href="https://colab.research.google.com/github/nataliepham6720/16-745_Optimal_Control/blob/main/wahba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import autograd.numpy as anp
from autograd import jacobian
import numpy as np

# Skew-symmetric matrix
def hat(v):
    return anp.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def L(q):
    s, v = q[0], q[1:4]
    return anp.block([
        [anp.array([[s]]), -v[None, :]],
        [v[:, None], s * anp.eye(3) + hat(v)]
    ])

def R(q):
    s, v = q[0], q[1:4]
    return anp.block([
        [anp.array([[s]]), -v[None, :]],
        [v[:, None], s * anp.eye(3) - hat(v)]
    ])

# Constants
T = anp.diag(anp.array([1.0, -1.0, -1.0, -1.0]))
H = anp.vstack([anp.zeros((1, 3)), anp.eye(3)])

def G(q):
    return L(q) @ H

def Q(q):
    return H.T @ R(q).T @ L(q) @ H

# Generate synthetic data
qtrue = np.random.randn(4)
qtrue = qtrue / np.linalg.norm(qtrue)
Qtrue = Q(qtrue)

vN = np.random.randn(3, 10)
vN /= np.linalg.norm(vN, axis=0, keepdims=True)
vB = anp.dot(Qtrue.T, vN)

def residual(q):
    return (vN - Q(q) @ vB).reshape(-1)

# Gauss-Newton loop
q = np.random.randn(4)
q = q / np.linalg.norm(q)

ϕ = anp.ones(3)
iter = 0

while anp.max(anp.abs(ϕ)) > 1e-8:
    r = residual(q)
    dr = jacobian(residual)(q)  # (30, 4)
    Grad_r = dr @ G(q)               # (30, 3)
    H = Grad_r.T @ Grad_r

    if anp.linalg.cond(H) > 1e8:
        print("Warning: ill-conditioned Hessian. Stopping.")
        break

    ϕ = -anp.linalg.solve(H, Grad_r.T @ r)
    dq = anp.concatenate([[anp.sqrt(1.0 - anp.dot(ϕ, ϕ))], ϕ])
    q = L(q) @ dq
    q = q / anp.linalg.norm(q)
    iter += 1

# Output results
print("Iterations:", iter)
print("q - qtrue:", q - qtrue)
print("q + qtrue:", q + qtrue)


Iterations: 0
q - qtrue: [-0.91000207  0.96543861 -0.99054699 -0.3767745 ]
q + qtrue: [-0.44579175 -0.72794843 -0.06427388 -0.6196062 ]


/usr/local/lib/python3.11/dist-packages/autograd/tracer.py:16: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")
